In [5]:
import pandas as pdb
import Reports

In [6]:
report_list = Reports.get_all_reports()

In [8]:
len(report_list)

10214

'Since our founding, the annual Company\nMeeting has been a marquee element of our culture. From presentations on our company\nstrategy to the inspiring stories shared by\nfaculty and students, the annual event\ngrounds everyone in why we’re here – to\neliminate the back row in education.'